# Query data by geometry

## Package imports

In [ ]:
import sys
import json

# installing needed packages - this may take a few minutes
try:
    import shapely.geometry
    import folium
except ModuleNotFoundError:
    !{sys.executable} -m pip install shapely
    !{sys.executable} -m pip install folium
    import shapely.geometry
    import folium

from pytreedb import db

## Import data

Specify the file where database is stored locally additionally to MongoDB.

In [ ]:
mydbfile = "my_query_pytree.db"

Define the (local) MongoDB connection and import data into pytreedb from URL.

In [ ]:
mydb = db.PyTreeDB(dbfile=mydbfile)
data_url = "https://github.com/3dgeo-heidelberg/pytreedb/raw/main/data/test/geojsons.zip"
mydb.import_data(data_url, overwrite=True)

## Get all trees inside a polygon

If we are only interested in trees in a certain area, we can query the tree database by a given geometry. For example, we can create a polygon and filter for trees lying within the polygon. Here, we are doing this for our forest plot "SP02" lying in the forest southwest of Bretten.

In [ ]:
SP02_coords = [
    [
        [8.657777973760469, 49.019526417240272],
        [8.6564204259407, 49.018633562408255],
        [8.655062878168346, 49.019526417240272],
        [8.656420425940698, 49.020419287904346],
        [8.657777973760469, 49.019526417240272],
    ]
]

poly_dict = json.dumps({"type": "Polygon", "coordinates": SP02_coords})

Let's first have a look at the polygon on the map.

In [ ]:
geom = json.loads(poly_dict)
search_geom = shapely.geometry.shape(geom)

m = folium.Map([49.00772676282502, 8.701981844725482], zoom_start=13)
folium.GeoJson(search_geom).add_to(m)
m

Now we query the database using the polygon.

In [ ]:
res = mydb.query_by_geometry(poly_dict)

To display the filtered trees on the map, we retrieve their locations, their IDs and their species.

In [ ]:
tree_locations = [tree["geometry"]["coordinates"] for tree in res]
tree_ids = [tree["properties"]["id"] for tree in res]
tree_species = [tree["properties"]["species"] for tree in res]

You can zoom in and click on the markers in the map created below to read out the ID and species of the respective tree.

In [ ]:
tooltip = "Click me!"
m = folium.Map([49.0195175, 8.6563631], zoom_start=17)
folium.GeoJson(search_geom).add_to(m)
for point, name, species in zip(tree_locations, tree_ids, tree_species):
    html = f"""
    <strong>{name}</strong><br>
    <em>{species}</em>
    """
    # note that we have to reverse latitude and longitude
    folium.Marker(point[1::-1], popup=html, tooltip=tooltip).add_to(m)
m

## Get all trees within a certain radius from a point

We can also query by a point geometry by specifying a distance around a point, within which trees must lie.

In [ ]:
query_point = [8.6995085, 49.0131634]
point_dict = json.dumps({"type": "Point", "coordinates": query_point})

search_radius = 150.0
res = mydb.query_by_geometry(point_dict, distance=search_radius)

tree_locations = [tree["geometry"]["coordinates"] for tree in res]
tree_ids = [tree["properties"]["id"] for tree in res]
tree_species = [tree["properties"]["species"] for tree in res]

In [ ]:
m = folium.Map([49.0131634, 8.6995085], zoom_start=17)
folium.Marker(
    query_point[::-1],
    tooltip=f"Center point, search radius = {search_radius}",
    icon=folium.Icon(color="black", icon="circle", prefix="fa"),
).add_to(m)
for point, name, species in zip(tree_locations, tree_ids, tree_species):
    html = f"""
    <strong>{name}</strong><br>
    <em>{species}</em>
    """
    # note that we have to reverse latitude and longitude
    folium.Marker(point[1::-1], popup=html, tooltip=tooltip).add_to(m)
m

## Using very large query geometries

When using query geometries whose area is greater than a single hemisphere, we have to use the custom MongoDB
coordinate system.

In [ ]:
huge_search_geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-100, 60],
            [-100, 0],
            [-100, -60],
            [100, -60],
            [100, 60],
            [-100, 60],
        ]
    ],
    "crs": {"type": "name", "properties": {"name": "urn:x-mongodb:crs:strictwinding:EPSG:4326"}},
}

m = folium.Map([49.00772676282502, 8.701981844725482], zoom_start=1)
folium.GeoJson(huge_search_geom).add_to(m)
m

In [ ]:
res = mydb.query_by_geometry(huge_search_geom)
print(len(res))

In [ ]:
tree_locations = [tree["geometry"]["coordinates"] for tree in res]
folium.Marker(tree_locations[0][1::-1]).add_to(m)
m